In [1]:
# !pip install -r requirements.txt
## Base of notebooks
%load_ext autoreload
%autoreload 2
import tensorflow as tf

In [2]:
from datetime import datetime

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
img_size = 128
task  = 'multidensity-allvs200'

In [3]:
import sys 
sys.path.append('..')

from utils import *
import models

In [4]:
base_data = stack_images(im_size=img_size, directory= "C:\\Users\\baran.tanriverdi\\Final Data")

100%|██████████| 43/43 [00:06<00:00,  6.39it/s]


Al for 5 Cannot found!


100%|██████████| 150/150 [00:16<00:00,  9.36it/s]

Fe2 : 51
Fe5 : 49
Fe10 : 100
Fe20 : 161
Fe200 : 153
Al2 : 43
Al10 : 16
Al20 : 150
Al200 : 136
Pb2 : 53
Pb5 : 50
Pb10 : 97
Pb20 : 150
Pb200 : 150
dict_keys(['Fe2', 'Fe5', 'Fe10', 'Fe20', 'Fe200', 'Al2', 'Al10', 'Al20', 'Al200', 'Pb2', 'Pb5', 'Pb10', 'Pb20', 'Pb200'])


In [5]:
np.asarray(base_data['Fe5']).shape

(49, 128, 128, 3)

In [6]:
# Configyure for current experiment

x1 = []
for imset in [
    base_data['Fe2'], 
    base_data['Fe5'], 
    base_data['Fe10'], 
    base_data['Fe20'], 
    base_data['Al2'], 
    base_data['Al10'], 
    base_data['Al20'], 
    base_data['Pb2'], 
    base_data['Pb5'], 
    base_data['Pb10'],
    base_data['Pb20']]:
    for im in imset:
        x1.append(im)
print(np.asarray(x1).shape)

x2 = []
for imset in [
    base_data['Fe200'],
    base_data['Al200'],
    base_data['Pb200']]:
    for im in imset:
        x2.append(im)
print(np.asarray(x2).shape)
        

imlist = []
one_hot_vector = np.eye(2)
for x in x1:
    imlist.append([x, one_hot_vector[0]])
for x in x2:
    imlist.append([x, one_hot_vector[1]])  
    
print(len(imlist), len(x1), len(x2))

data = np.asarray(imlist)

np.random.shuffle(data)

x = np.asarray([i[0] / 255.0 for i in data])
y = np.asarray([i[1] for i in data])

len(x), len(y)

boundary = int(len(x) * 0.75)

train_x = x[:boundary]
train_y = y[:boundary]
test_x = x[boundary:]
test_y = y[boundary:]

print(len(train_x), len(test_x))

(920, 128, 128, 3)
(439, 128, 128, 3)
1359 920 439


C:\Users\baran.tanriverdi\AppData\Local\Temp\ipykernel_5492\1521005721.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(imlist)


1019 340


In [8]:
histories = []
models_ = [
    models.unet(input_shape=(img_size, img_size, 3)), 
    models.vgg5_block(input_shape=(img_size, img_size, 3)),
    #models.vgg16(input_shape=(img_size, img_size, 3)), 
    #models.vgg19(input_shape=(img_size, img_size, 3)),
    #models.resnet50(input_shape=(img_size, img_size, 3)), 
    #models.deeplabv3plus(input_shape=(img_size, img_size, 3)),
    #models.nasnet_mobile(input_shape=(img_size, img_size, 3)), 
    #models.inception_resnetv2(input_shape=(img_size, img_size, 3)), 
    #models.double_unet(input_shape=(img_size, img_size, 3)),
]

for model in tqdm(models_):
    print(model.name)
    history =model.fit(train_x, train_y, batch_size=10,
                    validation_split=0.2,
                    epochs=25, 
                    #callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,),
                    verbose = 1)
    histories.append(history)
    MODEL_NAME = f"{date}_{model.name}_{img_size}_{task}"
    save_logs(history, MODEL_NAME)
        #save m-odel
    tf.keras.models.save_model(
        model,
        f'./models/{task}/{MODEL_NAME}.h5')
           

C:\Users\baran.tanriverdi\Anaconda3\envs\tf\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "vgg5_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_18 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_20 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                    

  0%|          | 0/2 [00:00<?, ?it/s]

unet
Epoch 1/25
82/82 [==============================] - 4156s 50s/step - loss: 87.6668 - accuracy: 0.6110 - f1_score: 0.5544 - cohen_kappa: -0.5613 - MatthewsCorrelationCoefficient: -0.0307 - auc: 0.6052 - val_loss: 0.6279 - val_accuracy: 0.6961 - val_f1_score: 0.5997 - val_cohen_kappa: -0.3972 - val_MatthewsCorrelationCoefficient: 0.1071 - val_auc: 0.7753
Epoch 2/25
82/82 [==============================] - 4112s 50s/step - loss: 0.6850 - accuracy: 0.6601 - f1_score: 0.5445 - cohen_kappa: -0.4150 - MatthewsCorrelationCoefficient: 0.0114 - auc: 0.6819 - val_loss: 0.6035 - val_accuracy: 0.6961 - val_f1_score: 0.5846 - val_cohen_kappa: -0.3721 - val_MatthewsCorrelationCoefficient: 0.0946 - val_auc: 0.7675
Epoch 3/25
82/82 [==============================] - 4093s 50s/step - loss: 0.6250 - accuracy: 0.6798 - f1_score: 0.5815 - cohen_kappa: -0.4340 - MatthewsCorrelationCoefficient: 0.1449 - auc: 0.7147 - val_loss: 0.5877 - val_accuracy: 0.6961 - val_f1_score: 0.5763 - val_cohen_kappa: -0.35

82/82 [==============================] - 710s 9s/step - loss: 15.8163 - accuracy: 0.6417 - f1_score: 0.5596 - cohen_kappa: -0.4865 - MatthewsCorrelationCoefficient: 0.0062 - auc: 0.6436 - val_loss: 0.6365 - val_accuracy: 0.6912 - val_f1_score: 0.5650 - val_cohen_kappa: -0.3534 - val_MatthewsCorrelationCoefficient: 0.0000e+00 - val_auc: 0.6912
Epoch 25/25
82/82 [==============================] - 710s 9s/step - loss: 0.6404 - accuracy: 0.6663 - f1_score: 0.5328 - cohen_kappa: -0.3851 - MatthewsCorrelationCoefficient: 0.0000e+00 - auc: 0.6744 - val_loss: 0.6250 - val_accuracy: 0.6912 - val_f1_score: 0.5650 - val_cohen_kappa: -0.3534 - val_MatthewsCorrelationCoefficient: 0.0000e+00 - val_auc: 0.6912


 50%|█████     | 1/2 [16:21:10<16:21:10, 58870.08s/it]

vgg5_block
Epoch 1/25
82/82 [==============================] - 51s 621ms/step - loss: 0.6867 - accuracy: 0.6356 - f1_score: 0.5928 - auc: 0.6433 - val_loss: 0.6604 - val_accuracy: 0.6961 - val_f1_score: 0.6882 - val_auc: 0.7993
Epoch 2/25
82/82 [==============================] - 50s 615ms/step - loss: 0.6334 - accuracy: 0.6724 - f1_score: 0.6266 - auc: 0.7041 - val_loss: 0.5830 - val_accuracy: 0.7206 - val_f1_score: 0.6823 - val_auc: 0.8127
Epoch 3/25
82/82 [==============================] - 51s 621ms/step - loss: 0.5988 - accuracy: 0.7018 - f1_score: 0.6811 - auc: 0.7512 - val_loss: 0.5418 - val_accuracy: 0.7696 - val_f1_score: 0.7459 - val_auc: 0.8283
Epoch 4/25
82/82 [==============================] - 51s 621ms/step - loss: 0.5969 - accuracy: 0.6748 - f1_score: 0.6535 - auc: 0.7437 - val_loss: 0.5428 - val_accuracy: 0.7206 - val_f1_score: 0.6470 - val_auc: 0.8043
Epoch 5/25
82/82 [==============================] - 51s 622ms/step - loss: 0.5567 - accuracy: 0.7043 - f1_score: 0.6882 -

100%|██████████| 2/2 [16:42:25<00:00, 30072.77s/it]   
